In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../src')

%load_ext autoreload
%autoreload 2

import torch
import torch_geometric
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

from torch_geometric.datasets import TUDataset
from preprocessing import data_transformation
from similarity import calculate_similarity_matrix

from model import GCN

In [2]:
dataset = TUDataset(root="/", name="MUTAG")

## Preprocessing

#### Split: Train test validation

```train_dataset```: for training model<br/>
```val_dataset```: evaluate model for hyperparameter tunning<br/>
```test_dataset```: testing model after complete training<br/>

In [3]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

In [4]:
tr, ts, vl = 0.8, 0.1, 0.1
dslen = len(dataset)
tri = round(tr*dslen)
tsi = round((tr+ts)*dslen)
train_dataset = dataset[:tri]
test_dataset = dataset[tri:tsi]
val_dataset = dataset[tsi:]

In [5]:
dataset.y

tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0])

In [6]:
print(len(train_dataset))
train_dataset.y

150


tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        1, 1, 0, 1, 1, 0])

In [7]:
len(test_dataset)
test_dataset.y

tensor([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1])

In [8]:
len(val_dataset)
val_dataset.y

tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0])

#### Batching

In [9]:
# paper 128
batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
print('train loader')
for data in train_loader:
    print(data.y)
    
print('val loader')
for data in val_loader:
    print(data.y)
    
print('test loader')
for data in test_loader:
    print(data.y)

train loader
tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1])
tensor([0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1])
tensor([0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0])
val loader
tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0])
test loader
tensor([1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1])


## Building Model

In [11]:
from torch_geometric.nn import GCNConv
from torch.nn import Linear

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_add_pool

### Base model

In [12]:
class Base(torch.nn.Module):
    # merging type: o --> complement only, s --> substraction, c --> concatenation
    def __init__(self, dataset, hidden_channels):
        super(Base, self).__init__()
        
        # weight seed
        torch.manual_seed(42)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        # classification layer
        
        self.lin = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # Embed original
        embedding = self.conv1(x, edge_index)
        embedding = embedding.relu()
        embedding = self.conv2(embedding, edge_index)
        embedding = embedding.relu()
        embedding = self.conv3(embedding, edge_index)
        embedding = embedding.relu()
        # subgraph_embedding = subgraph_embedding.relu()
        
        embedding = global_mean_pool(embedding, batch)
        h = self.lin(embedding)
        h = h.relu()
        h = self.lin2(h)
        
        return embedding, h

In [13]:
base = Base(dataset, 64)
base

Base(
  (conv1): GCNConv(7, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=64, bias=True)
  (lin2): Linear(in_features=64, out_features=2, bias=True)
)

In [88]:
def train_base(model, loader, experiment_mode=False):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
    
    model.train()
    
    for data in loader:
        if experiment_mode:
            emb, h = model(data.x, data.edge_index, data.batch, data.ptr)
        else:
            emb, h = model(data.x, data.edge_index, data.batch)
        loss = criterion(h, data.y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return loss
    #     print(h[0])
    # print(loss)

@torch.no_grad()
def test_base(model, loader, experiment_mode=False):
    model.eval()
    correct = 0
    for data in loader:
        if experiment_mode:
            emb, h = model(data.x, data.edge_index, data.batch, data.ptr)
        else:
            emb, h = model(data.x, data.edge_index, data.batch)
        pred = h.argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct/len(loader.dataset)

base = Base(dataset, 64)
train_base(base, train_loader)

tensor(0.7054, grad_fn=<NllLossBackward0>)

In [15]:
epoch = 100

base = Base(dataset, 64)
train_base(base, train_loader)

# Train
for _ in range(epoch):
    loss = round(train_base(base, train_loader).item(), 2)
    train_acc = round(test_base(base, train_loader), 2)
    val_acc = round(test_base(base, val_loader), 2)
    
    print(f'epoch {_}; loss: {loss}; train_acc: {train_acc}; test_acc: {val_acc}')

# Test
test = test_base(base, test_loader)
print(f'Accuracy: {test}')

epoch 0; loss: 0.69; train_acc: 0.34; test_acc: 0.37
epoch 1; loss: 0.69; train_acc: 0.34; test_acc: 0.37
epoch 2; loss: 0.69; train_acc: 0.34; test_acc: 0.37
epoch 3; loss: 0.69; train_acc: 0.51; test_acc: 0.42
epoch 4; loss: 0.69; train_acc: 0.7; test_acc: 0.53
epoch 5; loss: 0.69; train_acc: 0.74; test_acc: 0.63
epoch 6; loss: 0.69; train_acc: 0.67; test_acc: 0.63
epoch 7; loss: 0.69; train_acc: 0.66; test_acc: 0.63
epoch 8; loss: 0.69; train_acc: 0.66; test_acc: 0.63
epoch 9; loss: 0.69; train_acc: 0.66; test_acc: 0.63
epoch 10; loss: 0.7; train_acc: 0.66; test_acc: 0.63
epoch 11; loss: 0.7; train_acc: 0.66; test_acc: 0.63
epoch 12; loss: 0.7; train_acc: 0.66; test_acc: 0.63
epoch 13; loss: 0.7; train_acc: 0.66; test_acc: 0.63
epoch 14; loss: 0.7; train_acc: 0.66; test_acc: 0.63
epoch 15; loss: 0.7; train_acc: 0.66; test_acc: 0.63
epoch 16; loss: 0.7; train_acc: 0.66; test_acc: 0.63
epoch 17; loss: 0.7; train_acc: 0.66; test_acc: 0.63
epoch 18; loss: 0.7; train_acc: 0.66; test_acc:

### Experiment Model

In [16]:
from sklearn.cluster import AffinityPropagation

In [17]:
class Experiment(torch.nn.Module):
    # merging type: o --> complement only, s --> substraction, c --> concatenation
    def __init__(self, dataset, hidden_channels):
        super(Experiment, self).__init__()
        
        # weight seed
        torch.manual_seed(42)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        # classification layer
        
        self.lin = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # Embed original
        embedding = self.conv1(x, edge_index)
        embedding = embedding.relu()
        embedding = self.conv2(embedding, edge_index)
        
        # generate subgraph based on embeddings
        feature_emb = embedding.detach()
        G = data_transformation(edge_index, feature_emb)
        S = calculate_similarity_matrix(G)
        # clustering = AffinityPropagation(affinity='precomputed', random_state=123, max_iter=200).fit(S)
        
        embedding = global_mean_pool(embedding, batch)
        h = self.lin(embedding)
        h = h.relu()
        h = self.lin2(h)
        
        return embedding, h

    def data_transformation():
        print('s')
        


experiment = Experiment(dataset, 64)
# train_base(experiment, train_loader)

In [18]:
batch1 = None
for batch in train_loader:
    batch1 = batch
    break
print(batch1)
print(batch1.batch)
print("edge_index", batch1.edge_index)
print("batch",batch1.edge_attr)
print("ptr",batch1.ptr)

DataBatch(edge_index=[2, 2590], x=[1168, 7], edge_attr=[2590, 4], y=[64], batch=[1168], ptr=[65])
tensor([ 0,  0,  0,  ..., 63, 63, 63])
edge_index tensor([[   0,    0,    1,  ..., 1165, 1166, 1167],
        [   1,    5,    0,  ..., 1167, 1165, 1165]])
batch tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.]])
ptr tensor([   0,   17,   30,   43,   62,   73,  101,  117,  137,  149,  166,  183,
         203,  225,  238,  257,  279,  290,  307,  320,  338,  356,  373,  396,
         423,  440,  453,  476,  493,  516,  539,  561,  585,  608,  621,  638,
         652,  669,  684,  699,  712,  729,  742,  761,  778,  790,  813,  835,
         852,  872,  888,  914,  940,  959,  978,  992, 1009, 1030, 1055, 1078,
        1097, 1114, 1125, 1148, 1168])


In [19]:
print(max(batch1.edge_index[0]))
print(max(batch1.edge_index[1]))
print((dataset[0].edge_index))
print((batch1.ptr), '; len:', len(batch1.ptr))

tensor(1167)
tensor(1167)
tensor([[ 0,  0,  1,  1,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,  7,  7,
          8,  8,  8,  9,  9,  9, 10, 10, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14,
         15, 16],
        [ 1,  5,  0,  2,  1,  3,  2,  4,  9,  3,  5,  6,  0,  4,  4,  7,  6,  8,
          7,  9, 13,  3,  8, 10,  9, 11, 10, 12, 11, 13, 14,  8, 12, 12, 15, 16,
         14, 14]])
tensor([   0,   17,   30,   43,   62,   73,  101,  117,  137,  149,  166,  183,
         203,  225,  238,  257,  279,  290,  307,  320,  338,  356,  373,  396,
         423,  440,  453,  476,  493,  516,  539,  561,  585,  608,  621,  638,
         652,  669,  684,  699,  712,  729,  742,  761,  778,  790,  813,  835,
         852,  872,  888,  914,  940,  959,  978,  992, 1009, 1030, 1055, 1078,
        1097, 1114, 1125, 1148, 1168]) ; len: 65


In [20]:
graph_bound = {}

for i in range(len(batch1.ptr)-1):
    graph_bound[i] = [batch1.ptr[i].item(), batch1.ptr[i+1].item()]
    print(str(i)+".", batch1.ptr[i].item(), "-", batch1.ptr[i+1].item())

0. 0 - 17
1. 17 - 30
2. 30 - 43
3. 43 - 62
4. 62 - 73
5. 73 - 101
6. 101 - 117
7. 117 - 137
8. 137 - 149
9. 149 - 166
10. 166 - 183
11. 183 - 203
12. 203 - 225
13. 225 - 238
14. 238 - 257
15. 257 - 279
16. 279 - 290
17. 290 - 307
18. 307 - 320
19. 320 - 338
20. 338 - 356
21. 356 - 373
22. 373 - 396
23. 396 - 423
24. 423 - 440
25. 440 - 453
26. 453 - 476
27. 476 - 493
28. 493 - 516
29. 516 - 539
30. 539 - 561
31. 561 - 585
32. 585 - 608
33. 608 - 621
34. 621 - 638
35. 638 - 652
36. 652 - 669
37. 669 - 684
38. 684 - 699
39. 699 - 712
40. 712 - 729
41. 729 - 742
42. 742 - 761
43. 761 - 778
44. 778 - 790
45. 790 - 813
46. 813 - 835
47. 835 - 852
48. 852 - 872
49. 872 - 888
50. 888 - 914
51. 914 - 940
52. 940 - 959
53. 959 - 978
54. 978 - 992
55. 992 - 1009
56. 1009 - 1030
57. 1030 - 1055
58. 1055 - 1078
59. 1078 - 1097
60. 1097 - 1114
61. 1114 - 1125
62. 1125 - 1148
63. 1148 - 1168


Below --> Subgraph extractor with batch information

In [21]:
# idx_from = 0
# idx_to = 0
graph_counter = 0
graph_bound
edge_index = [[],[]]
Gs = []

from similarity import calculate_similarity_matrix, testt
# AP Clustering
from sklearn.cluster import AffinityPropagation

print(f'=== Graph {graph_counter} ===')
for i, (src, dst) in enumerate(zip(batch1.edge_index[0], batch1.edge_index[1])):
    # if (graph_counter < len(batch1.ptr)):
    lower_bound = graph_bound[graph_counter][0]
    upper_bound = graph_bound[graph_counter][1]
    if ((src >= lower_bound and src < upper_bound) or
        (dst >= lower_bound and dst < upper_bound)):
        # print(i,src.item()-lower_bound, dst.item()-lower_bound)
        edge_index[0].append(src - lower_bound)
        edge_index[1].append(dst - lower_bound)
    else:
        # print(edge_index)
        embs = []
        # make new graph
        for i, (b, emb) in enumerate(zip(batch1.batch, batch1.x)):
            if (b == graph_counter):
                # print(i, emb)
                embs.append(emb)
        
        G = data_transformation(edge_index, embs)
        Gs.append(G)
        print(sorted(list(G.nodes)))
        # print('pre', precalc_shortest_path_length)
        # for node in sorted(list(G.nodes)):
        #     print(G.nodes[node])
        
        
        # testt()
        if graph_counter == 10:
            print('masalah disini bro')
            break
        
        # Calculate S matrix
        S = calculate_similarity_matrix(G)
        
        # AP Clustering
        clustering = AffinityPropagation(affinity='precomputed', damping=0.9, random_state=123, max_iter=1000).fit(S)

        print(clustering.labels_)
        # print(clustering.)
        
        communities = {}
        print(set(clustering.labels_))
        # communities init
        for lab in clustering.labels_:
            communities[lab] = []
        
        for nd, clust in enumerate(clustering.labels_):
            communities[clust].append(nd)
        print("communities", communities) 
            
        edge_index = [[],[]]
        graph_counter+=1
        
        print(f'=== Graph {graph_counter} ===')
        
    if i == len(batch1.edge_index[0]) - 1:
        embs = []
        # make new graph
        for i, (b, emb) in enumerate(zip(batch1.batch, batch1.x)):
            if (b == graph_counter):
                # print(i, emb)
                embs.append(emb)
        
        G = data_transformation(edge_index, embs)
        Gs.append(G)
        
        S = calculate_similarity_matrix(G)
        # AP Clustering        
        clustering = AffinityPropagation(affinity='precomputed', damping=0.9, random_state=123, max_iter=1000).fit(S)
        
        print(sorted(list(G.nodes)))
        print(clustering.labels_)
        
        # print(edge_index)
        print('udh di akhir')

=== Graph 0 ===
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[0 0 0 0 1 1 1 3 3 3 3 2 3 3 2 2 2]
{0, 1, 2, 3}
communities {0: [0, 1, 2, 3], 1: [4, 5, 6], 3: [7, 8, 9, 10, 12, 13], 2: [11, 14, 15, 16]}
=== Graph 1 ===
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[2 1 1 1 0 0 1 0 2 2 2 2 2]
{0, 1, 2}
communities {2: [0, 8, 9, 10, 11, 12], 1: [1, 2, 3, 6], 0: [4, 5, 7]}
=== Graph 2 ===
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1 1 0 0 0 0 0 0 1 1 1 1 1]
{0, 1}
communities {1: [0, 1, 8, 9, 10, 11, 12], 0: [2, 3, 4, 5, 6, 7]}
=== Graph 3 ===
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
[1 0 1 0 1 1 2 2 3 3 3 2 2 4 4 3 4 4 4]
{0, 1, 2, 3, 4}
communities {1: [0, 2, 4, 5], 0: [1, 3], 2: [6, 7, 11, 12], 3: [8, 9, 10, 15], 4: [13, 14, 16, 17, 18]}
=== Graph 4 ===
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0 1 1 2 2 0 0 2 2 2 1]
{0, 1, 2}
communities {0: [0, 5, 6], 1: [1, 2, 10], 2: [3, 4, 7, 8, 9]}
=== Graph 5 ===
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15

In [22]:
dataset[3].x


tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.]])

In [23]:
for node in Gs[6].nodes:
    print(Gs[0].nodes[node])
    # Udah bisa tambah nodes per batch, tinggal masukin ke algo utama

{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([0., 0., 1., 0., 0., 0., 0.], dtype=float32)}
{'node_features': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)}
{'node_features': ar

In [24]:
dataset[63].edge_index

tensor([[ 0,  0,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  5,  5,  5,  6,  6,  6,
          7,  8,  9,  9,  9, 10, 11, 11, 11, 12, 12, 12, 13, 13, 14, 14, 15, 15,
         15, 16, 16, 17, 17, 17, 18, 19],
        [ 1,  5,  0,  2,  1,  3, 12,  2,  4,  9,  3,  5,  0,  4,  6,  5,  7,  8,
          6,  6,  3, 10, 11,  9,  9, 12, 16,  2, 11, 13, 12, 14, 13, 15, 14, 16,
         17, 11, 15, 15, 18, 19, 17, 17]])

In [25]:
for i, (b, emb) in enumerate(zip(batch1.batch, batch1.x)):
    if (b == 0):
        print(i, emb)

0 tensor([1., 0., 0., 0., 0., 0., 0.])
1 tensor([1., 0., 0., 0., 0., 0., 0.])
2 tensor([1., 0., 0., 0., 0., 0., 0.])
3 tensor([1., 0., 0., 0., 0., 0., 0.])
4 tensor([1., 0., 0., 0., 0., 0., 0.])
5 tensor([1., 0., 0., 0., 0., 0., 0.])
6 tensor([1., 0., 0., 0., 0., 0., 0.])
7 tensor([1., 0., 0., 0., 0., 0., 0.])
8 tensor([1., 0., 0., 0., 0., 0., 0.])
9 tensor([1., 0., 0., 0., 0., 0., 0.])
10 tensor([1., 0., 0., 0., 0., 0., 0.])
11 tensor([1., 0., 0., 0., 0., 0., 0.])
12 tensor([1., 0., 0., 0., 0., 0., 0.])
13 tensor([1., 0., 0., 0., 0., 0., 0.])
14 tensor([0., 1., 0., 0., 0., 0., 0.])
15 tensor([0., 0., 1., 0., 0., 0., 0.])
16 tensor([0., 0., 1., 0., 0., 0., 0.])


#### Model modification

In [26]:

# idx_from = 0
# idx_to = 0
graph_counter = 0
graph_bound
edge_index = [[],[]]
subgraph_edge_index = [[],[]]
Gs = []
sub_created = False
from similarity import calculate_similarity_matrix, testt
# AP Clustering
from sklearn.cluster import AffinityPropagation

batches = []
for b in train_loader:
    batches.append(b)
batch1 = batches[2]
print(batch1)

# return 0
graph_bound = {}

for i in range(len(batch1.ptr)-1):
    graph_bound[i] = [batch1.ptr[i].item(), batch1.ptr[i+1].item()]
    # print(str(i)+".", batch1.ptr[i].item(), "-", batch1.ptr[i+1].item())

for i, (src, dst) in enumerate(zip(batch1.edge_index[0], batch1.edge_index[1])):
    # if (graph_counter < len(batch1.ptr)):
    lower_bound = graph_bound[graph_counter][0]
    upper_bound = graph_bound[graph_counter][1]
    if ((src >= lower_bound and src < upper_bound) or
        (dst >= lower_bound and dst < upper_bound)):
        # print(i,src.item()-lower_bound, dst.item()-lower_bound)
        edge_index[0].append(src - lower_bound)
        edge_index[1].append(dst - lower_bound)
    else:
        sub_created = True
        # continue
        
        
        # # print(edge_index)
        # embs = []
        # # make new graph
        # for i, (b, emb) in enumerate(zip(batch1.batch, batch1.x)):
        #     if (b == graph_counter):
        #         # print(i, emb)
        #         embs.append(emb)
        
        # G = data_transformation(edge_index, embs)
        # Gs.append(G)
        # print(sorted(list(G.nodes)))
        # # print('pre', precalc_shortest_path_length)
        # # for node in sorted(list(G.nodes)):
        # #     print(G.nodes[node])
        
        
        # # testt()
        # if graph_counter == 10:
        #     print('masalah disini bro')
        #     break
        
        # # Calculate S matrix
        # S = calculate_similarity_matrix(G)
        
        # # AP Clustering
        # clustering = AffinityPropagation(affinity='precomputed', damping=0.9, random_state=123, max_iter=1000).fit(S)

        # print(clustering.labels_)
        # # print(clustering.)
        
        # communities = {}
        # print("cluster labels:", set(clustering.labels_))
        # # communities init
        # for lab in clustering.labels_:
        #     communities[lab] = []
        
        # for nd, clust in enumerate(clustering.labels_):
        #     communities[clust].append(nd)
        # print("communities", communities) 
            
        # edge_index = [[],[]]
        # graph_counter+=1
        
        # # make subgraph edge_index
        # for c in communities:
        #     w = G.subgraph(communities[c])
        #     # print("edges subgraph", w.edges)
        #     for sub in w.edges:
        #         # print(sub[0], sub[1])
        #         subgraph_edge_index[0].append(sub[0] + lower_bound)
        #         subgraph_edge_index[1].append(sub[1] + lower_bound)
                
        # print(subgraph_edge_index)
        # print(f'=== Graph {graph_counter} ===') 
        
    if (i == len(batch1.edge_index[0]) - 1) or sub_created:
        print(f'=== Graph {graph_counter} ===')
        
        sub_created = False
        
        embs = []
        # make new graph
        for i, (b, emb) in enumerate(zip(batch1.batch, batch1.x)):
            if (b == graph_counter):
                # print(i, emb)
                embs.append(emb)
        
        G = data_transformation(edge_index, embs)
        print(G)
        print("edge_idx", edge_index)
        print(max(G.nodes))
        print(len(G.nodes))
        
        Gs.append(G)
        
        S = calculate_similarity_matrix(G)
        # AP Clustering        
        clustering = AffinityPropagation(affinity='precomputed', damping=0.9, random_state=123, max_iter=1000).fit(S)
        
        print(sorted(list(G.nodes)))
        print(clustering.labels_)
        
        # Modif disini nanti
        #########
        communities = {}
        # print("cluster labels:", set(clustering.labels_))
        # communities init
        for lab in clustering.labels_:
            communities[lab] = []
        
        for nd, clust in enumerate(clustering.labels_):
            communities[clust].append(nd)
        # print("communities", communities) 
            
        edge_index = [[],[]]
        graph_counter+=1
        
        # make subgraph edge_index
        for c in communities:
            w = G.subgraph(communities[c])
            # print("edges subgraph", w.edges)
            for sub in w.edges:
                # print(sub[0], sub[1])
                subgraph_edge_index[0].append(sub[0] + lower_bound)
                subgraph_edge_index[1].append(sub[1] + lower_bound)
                
        print(subgraph_edge_index)
        if (graph_counter == 10):
            break
        # print(f'=== Graph {graph_counter} ===')
        # print(edge_index)
        # print('udh di akhir')

embeddings_used = []
nodes_used = set(np.array(subgraph_edge_index).flatten())
feat = batch1.x
# for i, f in enumerate(feat):
#     if(i in nodes_used):
#         embeddings_used.append(feat[i].detach().numpy())
        
# print(len(embeddings_used))
# z = torch.zeros(len(embeddings_used), len(embeddings_used[0]))
# z = torch.tensor(embeddings_used)

# print(torch.tensor(subgraph_edge_index).size())
# nodes_used = set(np.array(subgraph_edge_index).flatten())
# feat = batch1.x
# for i, f in enumerate(feat):
#     if(i in nodes_used):
#         print(i)

DataBatch(edge_index=[2, 720], x=[337, 7], edge_attr=[720, 4], y=[22], batch=[337], ptr=[23])
=== Graph 0 ===
Graph with 14 nodes and 14 edges
edge_idx [[tensor(0), tensor(0), tensor(1), tensor(1), tensor(2), tensor(2), tensor(2), tensor(3), tensor(3), tensor(3), tensor(4), tensor(4), tensor(5), tensor(5), tensor(5), tensor(6), tensor(6), tensor(6), tensor(7), tensor(8), tensor(9), tensor(9), tensor(9), tensor(10), tensor(11), tensor(12), tensor(12), tensor(13)], [tensor(1), tensor(5), tensor(0), tensor(2), tensor(1), tensor(3), tensor(12), tensor(2), tensor(4), tensor(9), tensor(3), tensor(5), tensor(0), tensor(4), tensor(6), tensor(5), tensor(7), tensor(8), tensor(6), tensor(6), tensor(3), tensor(10), tensor(11), tensor(9), tensor(9), tensor(2), tensor(13), tensor(12)]]
13
14
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[0 0 2 1 1 0 0 0 0 1 1 1 2 2]
[[0, 0, 5, 6, 6, 2, 12, 3, 3, 9, 9], [1, 5, 6, 7, 8, 12, 13, 4, 9, 10, 11]]
=== Graph 1 ===
Graph with 11 nodes and 11 edges
edge_idx 

In [94]:

from similarity import calculate_similarity_matrix, testt


# AP Clustering
from sklearn.cluster import AffinityPropagation

from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_max_pool

import torch.nn.functional as F

class Experiment(torch.nn.Module):
    # merging type: o --> complement only, s --> substraction, c --> concatenation
    def __init__(self, dataset, hidden_channels):
        super(Experiment, self).__init__()
        
        # weight seed
        torch.manual_seed(42)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        
        # embeddings for subgraph
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.conv5 = GCNConv(hidden_channels, hidden_channels)
        # self.conv6 = GCNConv(hidden_channels, hidden_channels)
        # classification layer
        
        self.lin = Linear(hidden_channels*2, hidden_channels)
        self.lin2 = Linear(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index, batch, ptr):
        # Embed original
        embedding = self.conv1(x, edge_index)
        embedding = embedding.relu()
        embedding = self.conv2(embedding, edge_index)
        embedding = embedding.relu()
        embedding = self.conv3(embedding, edge_index)
        embedding = embedding.relu()
        
        # generate subgraph based on embeddings
        feature_emb = embedding.detach()
        # G = data_transformation(edge_index, feature_emb)
        # S = calculate_similarity_matrix(G)
        # clustering = AffinityPropagation(affinity='precomputed', random_state=123, max_iter=200).fit(S)
        subgraph_edge_index, _ = self.subgraph_generator(feature_emb, edge_index, batch, ptr)
        subgraph_embedding = self.conv4(embedding, subgraph_edge_index)
        subgraph_embedding = subgraph_embedding.relu()
        subgraph_embedding = self.conv5(subgraph_embedding, subgraph_edge_index)
        subgraph_embedding = subgraph_embedding.relu()
        
        # subgraph_embedding = self.conv1(x, subgraph_edge_index)
        # subgraph_embedding = subgraph_embedding.relu()
        # subgraph_embedding = self.conv2(subgraph_embedding, subgraph_edge_index)
        # subgraph_embedding = subgraph_embedding.relu()
        # subgraph_embedding = self.conv3(subgraph_embedding, subgraph_edge_index)
        # subgraph_embedding = subgraph_embedding.relu()
        
        # print(subgraph_edge_index)
        embedding = global_mean_pool(embedding, batch)
        # self.subgraph_pooling("","","")
        subgraph_embedding = global_max_pool(subgraph_embedding, batch)
        
        
        h = torch.cat((embedding, subgraph_embedding), 1)
        
        h = F.dropout(h, p=0.3, training=self.training)
        h = self.lin(h)
        h = h.relu()
        x = F.dropout(h, p=0.3, training=self.training)
        h = self.lin2(h)
        
        return embedding, h
    
    def subgraph_pooling(self, embeddings, batch, ptr):
        print('subgraph pooling')

    def subgraph_generator(self, embeddings, batch_edge_index, batch, ptr):
        '''
        Return subgraph_edge_index (edge_index of created subgraph)
        '''
        # print('processing subgraph_generator...')
        graph_counter = 0
        edge_index = [[],[]]
        subgraph_edge_index = [[],[]]
        # Gs = []
        sub_created = False
        graph_bound = {}

        for i in range(len(ptr)-1):
            graph_bound[i] = [ptr[i].item(), ptr[i+1].item()]
        
        for i, (src, dst) in enumerate(zip(batch_edge_index[0], batch_edge_index[1])):
            lower_bound = graph_bound[graph_counter][0]
            upper_bound = graph_bound[graph_counter][1]
            if ((src >= lower_bound and src < upper_bound) or
                (dst >= lower_bound and dst < upper_bound)):
                
                edge_index[0].append(src - lower_bound)
                edge_index[1].append(dst - lower_bound)
            else:
                sub_created = True
                
            if (i == len(batch_edge_index[0]) - 1) or sub_created:
                # print(f'=== Graph {graph_counter} ===')
                
                sub_created = False
                
                embs = []
                # make new graph
                for i, (b, emb) in enumerate(zip(batch, embeddings)):
                    if (b == graph_counter):
                        embs.append(emb)
                
                G = data_transformation(edge_index, embs)
                # dont need this at the moment
                # Gs.append(G)
                
                # Calculate similarity matrix
                S = calculate_similarity_matrix(G)
                
                # AP Clustering        
                # clustering = AffinityPropagation(affinity='precomputed', damping=0.9, random_state=123, max_iter=1000).fit(S)
                clustering = AffinityPropagation(affinity='precomputed', damping=0.9, random_state=123, convergence_iter=5, max_iter=100).fit(S)
                
                # Get community
                communities = {}
                for lab in clustering.labels_:
                    communities[lab] = []
                
                for nd, clust in enumerate(clustering.labels_):
                    communities[clust].append(nd)
                
                edge_index = [[],[]]
                graph_counter+=1
                
                # Make subgraph edge_index
                for c in communities:
                    w = G.subgraph(communities[c])
                    for sub in w.edges:
                        subgraph_edge_index[0].append(sub[0] + lower_bound)
                        subgraph_edge_index[1].append(sub[1] + lower_bound)
                        
                # INI LUPA WOY
                # if (graph_counter == 10):
                #     break
                
                
        # print("finished subgraph_generator")
        
        # embeddings_used = []
        # nodes_used = set(np.array(subgraph_edge_index).flatten())
        
        # for i, f in enumerate(embeddings):
        #     if(i in nodes_used):
        #         embeddings_used.append(embeddings[i].detach().numpy())
                
        
        # print("nodes used", len(nodes_used))
        # print("nodes used", (nodes_used))
        # z = torch.tensor(embeddings_used)
        # print(z.size())
        
        # return torch.tensor(subgraph_edge_index), z
        # pakai embeddings yang awal
        return torch.tensor(subgraph_edge_index), torch.tensor(embeddings)
    # (embeddings)
    
btch = None
experiment = Experiment(dataset, 64)
bcount = 0
for b in train_loader:
    # print("batch count",bcount)
    bcount+=1
    btch = b
    # print(btch.ptr)
    experiment(btch.x, btch.edge_index, btch.batch, btch.ptr)
    break
    # break
    # experiment(btch.x, btch.edge_index, btch.batch)

# print(experiment)
# experiment(btch.x, btch.edge_index, btch.batch)

In [96]:
def expTrain(train_loader, val_loader, test_loader, epoch = 2):
    import warnings
    warnings.filterwarnings("ignore", category=UserWarning) 

    experiment = Experiment(dataset, 64)

    # Train
    print('process training')
    for _ in range(epoch):
        loss = round(train_base(experiment, train_loader, True).item(), 5)
        train_acc = round(test_base(experiment, train_loader, True), 5)
        val_acc = round(test_base(experiment, val_loader, True), 5)
        
        print(f'epoch {_}; loss: {loss}; train_acc: {train_acc}; test_acc: {val_acc}')

    # Test
    print('process testing')
    test = test_base(experiment, test_loader, True)
    print(f'Accuracy: {test}')

# expTrain(train_loader, val_loader, test_loader, epoch = 1)

In [97]:
def baseTrain(train_loader, val_loader, test_loader, epoch = 10):
    base = Base(dataset, 64)

    # Train
    for _ in range(epoch):
        loss = round(train_base(base, train_loader).item(), 5)
        train_acc = round(test_base(base, train_loader), 5)
        val_acc = round(test_base(base, val_loader), 5)
        
        print(f'epoch {_}; loss: {loss}; train_acc: {train_acc}; val_acc: {val_acc}; test: {round(test_base(base, test_loader), 2)}')

    # Test
    test = test_base(base, test_loader)
    print(f'Accuracy: {test}')

#### Cross validation 10

In [62]:
from sklearn.model_selection import KFold

In [93]:
# 
train_dataset
test_dataset
k = 10

splits = KFold(n_splits=k,shuffle=True,random_state=42)
k_counter = 0

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(train_dataset)))):
    # print('Fold {}'.format(fold + 1))
    # print(f'Fold',fold,'Train_idx',train_idx,'Val_idx',val_idx)
    print(f'Fold {fold}/{k}')
    if k_counter > 2:
        break
    
    fold_train = []
    for key in train_idx:
        fold_train.append(dataset[key])

    fold_val = [] 
    for key in val_idx:
        fold_val.append(dataset[key])

    tr = DataLoader(fold_train, batch_size=batch_size, shuffle=False)
    vd = DataLoader(fold_val, batch_size=batch_size, shuffle=False)
    ts = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Base model
    print("=== Base model ===")
    baseTrain(tr, vd, ts, 10)
    print("=== Experiment model ===")
    expTrain(tr, vd, ts, 10)
    
    k_counter += 1

Fold 0/10
=== Base model ===
epoch 0; loss: 0.69; train_acc: 0.33; val_acc: 0.4; test: 0.26
epoch 1; loss: 0.71; train_acc: 0.67; val_acc: 0.6; test: 0.74
epoch 2; loss: 0.7; train_acc: 0.67; val_acc: 0.6; test: 0.74
epoch 3; loss: 0.7; train_acc: 0.67; val_acc: 0.6; test: 0.74
epoch 4; loss: 0.7; train_acc: 0.67; val_acc: 0.6; test: 0.74
epoch 5; loss: 0.7; train_acc: 0.67; val_acc: 0.6; test: 0.74
epoch 6; loss: 0.7; train_acc: 0.67; val_acc: 0.6; test: 0.74
epoch 7; loss: 0.7; train_acc: 0.67; val_acc: 0.6; test: 0.74
epoch 8; loss: 0.7; train_acc: 0.67; val_acc: 0.6; test: 0.74
epoch 9; loss: 0.69; train_acc: 0.67; val_acc: 0.6; test: 0.74
Accuracy: 0.7368421052631579
Fold 1/10
=== Base model ===
epoch 0; loss: 0.7; train_acc: 0.34; val_acc: 0.33; test: 0.26
epoch 1; loss: 0.97; train_acc: 0.66; val_acc: 0.67; test: 0.74
epoch 2; loss: 0.87; train_acc: 0.66; val_acc: 0.67; test: 0.74
epoch 3; loss: 0.81; train_acc: 0.66; val_acc: 0.67; test: 0.74
epoch 4; loss: 0.81; train_acc: 0.6